# An introduction to unfairness in ML

In this exercise, you will learn about some basic machine learning tasks and the fairness issues that could arise there.

## Installing the necessary libraries

In [ ]:
pip install pandas folktables requests scikit-learn --quiet

# Exercise 1: Training risk prediction models

In this exercise, we will work with the ProPublic COMPAS dataset and train a risk prediction model.

#### A quick intro to the dataset
ProPublica, a news organization, compiled a list of all criminal oﬀenders screened through the COMPAS (Correctional Oﬀender Management Profiling for Alternative Sanctions) tool 5 in Broward County, Florida during 2013-2014. The data includes information on the defendents’ demographic features (gender, race, age), criminal history (charge for which the person was arrested, number of prior oﬀenses) and the risk score assigned to the oﬀender by COMPAS. ProPublica also collected the ground truth on whether or not these individuals actually recidivated within two years after the screening. To learn more about the dataset, see [this article](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm).

#### Your task
Your task will be to inspect this data and train a risk prediction model on it.

## Exercise 1a: Download the data and select your features [20 mins]

The code below downloads the COMPAS dataset for you. You will be predicting whether a defendent will recidivate (commit a crime within next two year). The corresponding label is `two_year_recid`.

The COMPAS tool also trains a score ranging from 1 to 10 assigning the recidivism risk. We will train our own risk model and compare it against the COMPAS score.

Inspect the data and identify the features that you would like to use for training your risk prediction model. Create a new pandas dataframe that only contains the features that you are interested in. This will be your feature vector $\mathbf{x}$.

In [ ]:
import folktables
from folktables import ACSDataSource, ACSEmployment
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
from pathlib import Path
import requests


# Based on: https://fairlens.readthedocs.io/en/latest/user_guide/compas.html
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
local_name = Path("compas-scores-two-years.csv")
if not local_name.is_file():
    response = requests.get(url)
    with open("compas-scores-two-years.csv", "w") as f:
        f.write(response.content.decode("utf-8"))
df = pd.read_csv(local_name)
df = df.sample(frac=1, random_state=1)

df = df[(df["days_b_screening_arrest"] <= 30)
        & (df["days_b_screening_arrest"] >= -30)
        & (df["is_recid"] != -1)
        & (df["c_charge_degree"] != 'O')
        & (df["score_text"] != 'N/A')].reset_index(drop=True)

In [ ]:
# Inspect the dataframe
pd.set_option('display.max_columns', None)
df

In [ ]:
# Your code for separating out the features here
df.columns # List all the features

In [ ]:
import matplotlib.pyplot as plt

selected_cols = [  # The features that we will use for classification
    "age",
    "juv_fel_count",
    "juv_misd_count",
    "juv_other_count",
    "priors_count",
    "c_charge_degree",
]
sens = "race"  #  Will not use for classification but use it for fairness analysis
label = "two_year_recid"  # The label we will try to predict

df_selected = df[selected_cols + [sens] + [label]]
df_selected

In [ ]:
import seaborn as sns

for col in selected_cols:
    plt.figure()
    sns.histplot(df_selected, x=col, stat="probability", hue="race")

## Exercise 1b: Training and evaluating the risk predictors [25 mins]

Train a scikit learn [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model to predict the risk scores. Recall that the Logistic regression model assigns a probability of positive class as:
$$
p(y=1 | x) = 1 / [1 + \exp(-d(\mathbf{x}))]
$$

where $d(\mathbf{x})$ is the distance from the decision boundary.

You can access this probability using the [predict_proba](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict_proba) method of the model.

Your tasks are:
1. Evaluate your risk prediction model based on accuracy, TPR, TNR, PPV, NPV and AUROC. Use a threshold of $0.5$ to binarize your decisions.
2. Evaluate the COMPAS risk prediction model (in the column `v_decile_score`) with the same metrics. Use a threshold of $5$ to binarize the decisions. How do the two scores compare?

In [ ]:
y = df_selected[label].to_numpy()
z = df_selected[sens].to_numpy()
df_selected.drop(columns=[label, sens], inplace=True)
df_one_hot = pd.get_dummies(df_selected)
x = df_one_hot.to_numpy()
x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, random_state=1122)

model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_risk = model.predict_proba(x_test)[:,1]

# Exercise 2: Measuring unfairness (25 mins)
Let us go back to the model we trained for the COMPAS data. Compute all the fairness metrics from the lecture slides. For computing fairness, consider the groups white and black. Can you think of ways to remove this unfairness?

In [ ]:
import sklearn

def compute_model_stats(y, y_pred, y_risk):
    tp = np.logical_and(y==1, y_pred==1).sum()
    tn = np.logical_and(y==0, y_pred==0).sum()
    fp = np.logical_and(y==0, y_pred==1).sum()
    fn = np.logical_and(y==1, y_pred==0).sum()
    return {
        "pos_frac": (y_pred==1).mean(),
        "acc": (y==y_pred).mean(),
        "tpr": tp / (tp + fn),
        "tnr": tn / (tn + fp),
        "ppv": tp / (tp + fp),
        "npv": tn / (tn + fn),
        "auc": sklearn.metrics.roc_auc_score(y, y_risk),
    }


perf_overall = compute_model_stats(y_test, y_pred, y_risk)
perf_white = compute_model_stats(y_test[z_test=="Caucasian"], y_pred[z_test=="Caucasian"], y_risk[z_test=="Caucasian"])
perf_black = compute_model_stats(y_test[z_test=="African-American"], y_pred[z_test=="African-American"], y_risk[z_test=="African-American"])

print(f"| {'metric':<10} |  All |  B   |   W  |")
for k in perf_overall.keys():
    print(f"| {k:<10} | {perf_overall[k]:0.2f} | {perf_black[k]:0.2f} | {perf_white[k]:0.2f} |")
    

## Exercise 3: Transferability of risk scores (20 mins)

You might recall the ACS employment data from the last lecture. We were trying to predict if someone is employed.

In this exercise, we will measure how transferable the risk scores are over different states. Below is the code to train download the data for the state of Alabama (AL). Train a model and compute its accuracy (you did that in the last exercise too).

How well does the model trained on AL generalize to different states like CA and TX?

---
Just to remind you, the features in the dataset are:

 * AGEP (Age)
 * SCHL (Educational attainment)
 * MAR (Marital status)
 * SEX (Sex): 1 denotes Male and 2 Female
 * DIS (Disability recode): 1 denotes a disability and 2 a disability
 * ESP (Employment status of parents)
 * MIG (Mobility status (lived here 1 year ago)
 * CIT (Citizenship status)
 * MIL (Military service)
 * ANC (Ancestry recode)
 * NATIVITY (Nativity)
 * RELP (Relationship)
 * DEAR (Hearing diﬃculty)
 * DEYE (Vision diﬃculty)
 * DREM (Cognitive diﬃculty)
 * RAC1P (Recoded detailed race code): (1 means white alone, 2 means Black or African American alone)
 * GCL (Grandparents living with grandchildren)


For more details of the precise feature values, see Appendix B4 of the paper (https://arxiv.org/pdf/2108.04884)

---

In [ ]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)  # Limiting to AL. You can try another state or all the states.
x, y, group = ACSEmployment.df_to_numpy(acs_data)  # The group in this case is the race. It is also included in the features.

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print(f"Test accuracy on AL: {(y_pred==y_test).mean(): 0.2f}")

In [ ]:
# Your code here

def print_performance(y_true, y_pred):
    print(f"Accuracy: {(y_true==y_pred).mean(): 0.2f}")
    print("Confusion matrix")
    with np.printoptions(precision=2):
        print(sklearn.metrics.confusion_matrix(y_true, y_pred, normalize="true"))

def test_old_model(old_model, target_state):
    x, y, group = ACSEmployment.df_to_numpy(data_source.get_data(states=[target_state], download=True))
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

    own_model = LogisticRegression(max_iter=1000)
    own_model.fit(x_train, y_train)

    y_pred_own = own_model.predict(x_test)
    y_pred_old = old_model.predict(x_test)

    print(f"\n== AL model performance on {target_state} data")
    print_performance(y_test, y_pred_old)
    print(f"\n== {target_state} model performance on {target_state} data")
    print_performance(y_test, y_pred_own)

test_old_model(model, "CA")
print("---------")
test_old_model(model, "TX")